In [66]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/data-kiemtra-1/data-kiem-tra-1.csv


In [67]:
import numpy as np
import pandas as pd

def generate_data(n=100, seed=42):
    """
    Generate synthetic time series data with exogenous variables.
    
    Parameters:
    n (int): Number of data points.
    seed (int): Seed for reproducibility.
    
    Returns:
    pd.DataFrame: DataFrame containing the endogenous and exogenous variables.
    """
    np.random.seed(seed)
    Y = np.cumsum(np.random.randn(n))  # Random walk
    X1 = np.random.randn(n)
    X2 = np.random.randn(n)
    data = pd.read_csv('/kaggle/input/data-kiemtra-1/data-kiem-tra-1.csv')({'Y': Y, 'X1': X1, 'X2': X2}, index=pd.date_range(start='2020-01-01', periods=n))
    #data = pd.DataFrame({'Y': Y, 'X1': X1, 'X2': X2}, index=pd.date_range(start='2020-01-01', periods=n))
    return data

In [ ]:
data = pd.read_csv('/kaggle/input/data-kiemtra-1/data-kiem-tra-1.csv')

In [ ]:
print(df.head())

In [ ]:
# Chuyển đổi cột 'date' thành định dạng datetime
df['date'] = pd.to_datetime(df['date'])

# Đặt cột 'date' làm chỉ mục
df.set_index('date', inplace=True)

# Chọn các biến ngoại sinh
exog = df[['x1', 'x2']]  # Thay đổi tên cột theo dữ liệu thực tế của bạn

# Chia dữ liệu thành tập huấn luyện và kiểm tra
train_size = int(len(df) * 0.8)
train, test = df[:train_size], df[train_size:]
exog_train, exog_test = exog[:train_size], exog[train_size:]

### ARIMAX model

In [69]:
from statsmodels.tsa.statespace.sarimax import SARIMAX

def fit_arimax(data, order=(1, 1, 1), exog_cols=['truong_3','truong_4'], forecast_steps=10):
    """
    Fit an ARIMAX model to the data and make forecasts.
    
    Parameters:
    data (pd.DataFrame): DataFrame containing the time series and exogenous variables.
    order (tuple): The (p,d,q) order of the ARIMA model.
    exog_cols (list): List of column names for the exogenous variables.
    forecast_steps (int): Number of steps to forecast.
    
    Returns:
    pd.DataFrame: DataFrame containing the observed, forecasted values, and confidence intervals.
    """
    exog = data[exog_cols]
    model = SARIMAX(data['truong_3'], exog=exog, order=order)
    results = model.fit()
    print(results.summary())

    forecast = results.get_forecast(steps=forecast_steps, exog=np.random.randn(forecast_steps, len(exog_cols)))
    forecast_index = pd.date_range(start=data.index[-1] + pd.Timedelta(days=1), periods=forecast_steps)
    forecast_mean = forecast.predicted_mean
    forecast_ci = forecast.conf_int()

    forecast_df = pd.DataFrame({'Forecast': forecast_mean}, index=forecast_index)
    forecast_df['Lower CI'] = forecast_ci.iloc[:, 0]
    forecast_df['Upper CI'] = forecast_ci.iloc[:, 1]
    
    return forecast_df

In [70]:
import matplotlib.pyplot as plt

def plot_results(data, forecast_df):
    """
    Plot the observed data and forecasted values.
    
    Parameters:
    data (pd.DataFrame): DataFrame containing the observed time series.
    forecast_df (pd.DataFrame): DataFrame containing the forecasted values and confidence intervals.
    """
    plt.figure(figsize=(12, 6))
    plt.plot(data.index, data['Y'], label='Observed')
    plt.plot(forecast_df.index, forecast_df['Forecast'], label='Forecast')
    plt.fill_between(forecast_df.index, forecast_df['Lower CI'], forecast_df['Upper CI'], color='pink', alpha=0.3)
    plt.legend()
    plt.title('ARIMAX Model Forecast')
    plt.show()


In [ ]:
# Generate synthetic data
data = generate_data()

# Fit ARIMAX model and forecast
forecast_df = fit_arimax(data, order=(1, 1, 1), forecast_steps=10)

# Plot the results
plot_results(data, forecast_df)

 This problem is unconstrained.


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            5     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f= -1.05939D+01    |proj g|=  7.91245D+10


/opt/conda/lib/python3.10/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "

 Line search cannot locate an adequate point after MAXLS
  function and gradient evaluations.
  Previous x, f and g restored.
 Possible causes: 1 error in function or gradient evaluation;
                  2 rounding error dominate computation.



           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    5      1     21      1     0     0   7.912D+10  -1.059D+01
  F =  -10.593943645568773     

ABNORMAL_TERMINATION_IN_LNSRCH                              
